In [9]:
%pip install --pre unified-planning

Note: you may need to restart the kernel to use updated packages.


In [13]:
from unified_planning.shortcuts import *

plotting_problem = unified_planning.model.Problem('plotting_problem')

Color = UserType('Color')
red = Object('red', Color)
blue = Object('blue', Color)
green = Object('green', Color)
black = Object('black', Color)
yellow = Object('yellow', Color)
none = Object('none', Color)
plotting_problem.add_objects([red, blue, green, black, yellow, none])

blocks = Fluent('blocks', ArrayType(3, ArrayType(5, Color)))
plotting_problem.add_fluent(blocks, default_initial_value=[
    [none, green, none, green, black],
    [blue, red, none, red, blue],
    [blue, blue, none, blue, red]
])
hand = Fluent('hand', Color)
plotting_problem.add_fluent(hand, default_initial_value=yellow)

rows = blocks.type.size
columns = blocks.type.elements_type.size

############################################################################################################################################
#                                                                 ACTIONS                                                                  #
############################################################################################################################################

################################################################ SHOOT ROW #################################################################
shoot_row = unified_planning.model.InstantaneousAction('shoot_row', p=Color, r=IntType(0, rows-1), l=IntType(0, columns-2))
# p: color of the blocks that we want to eliminate
# r: row
# l: last column affected (cannot be the last one) - next block is different than p and not none
p = shoot_row.parameter('p')
r = shoot_row.parameter('r')
l = shoot_row.parameter('l')

      # Preconditions
# p is the color of the blocks that we want to eliminate
shoot_row.add_precondition(Not(Equals(p, yellow)))
# p is the same colour as the hand or the hand is yellow
shoot_row.add_precondition(Or(Equals(p, hand), Equals(yellow, hand)))
# the last's following block is not none and different than p
shoot_row.add_precondition(And(Not(Equals(blocks[r][l+1], p)), Not(Equals(blocks[r][l+1], none))))
# the blocks before the last are none or p
before = []
for c in range(0, columns):
    before.append(Or(GT(c,l), Equals(blocks[r][c], p), Equals(blocks[r][c], none)))
shoot_row.add_precondition(And(before))

      # Effects
# the hand saves the colour of the last's following block
shoot_row.add_effect(hand, blocks[r][l+1])
# the last's following block saves the colour of the first block
shoot_row.add_effect(blocks[r][l+1], p)
# changes for each column
for c in range(0, columns):
    # the last's previous blocks with colour p become its above block or none if it doesn't have
    condition_same_colour = And(LE(c,l), Equals(blocks[r][c], p))
    for a in range(0, rows):
        if a == 0:
            shoot_row.add_effect(blocks[a][c], none, And(condition_same_colour))
        else:
          shoot_row.add_effect(blocks[a][c], blocks[a-1][c], And(condition_same_colour, LE(a,r)))

# acabat!!

################################ SHOOT FULL ROW #####################################
shoot_full_row = unified_planning.model.InstantaneousAction('shoot_full_row', p=Color, r=IntType(0, rows-1))
# p: color of the blocks that we want to eliminate
# r: row
p = shoot_full_row.parameter('p')
r = shoot_full_row.parameter('r')

    # Preconditions
# p is the color of the blocks that we want to eliminate
shoot_full_row.add_precondition(Not(Equals(p, yellow)))
# p is the same colour as the hand or the hand is yellow
shoot_full_row.add_precondition(Or(Equals(p, hand), Equals(hand, yellow)))
# all the blocks of the row are none or p
before = []
for c in range(0, columns):
    before.append(Or(Equals(blocks[r][c], p), Equals(blocks[r][c], none)))
shoot_full_row.add_precondition(And(before))

    # Effects
# changes for each column
for c in range(0, columns):
    # the last's previous blocks with colour p become its above block or none
    condition_same_colour = And(Equals(blocks[r][c], p))
    condition_none = And(Equals(blocks[r][c], none))
    
    if c == columns-1:
        # si es condition_none => no te cap a sobre !!
        same = []
        for u in range(0, rows-1):
            condition_under = GT(u,r)
            condition_next_different = And(Not(Equals(blocks[u+1][c], p)), Not(Equals(blocks[u][c], none)))
            same.append(Or(Equals(blocks[u][c], p), Equals(blocks[u][c], none)))
            shoot_full_row.add_effect(blocks[u][c], none, And(condition_under, And(same), condition_none)) 
            shoot_full_row.add_effect(blocks[u+1][c], p, And(condition_under, And(same), condition_none, condition_next_different))
            shoot_full_row.add_effect(hand, blocks[u+1][c], And(condition_under, And(same), condition_none, condition_next_different)) 

            
    # la resta de columnes previes nomes canvien els de a dalt!
    else:
        for a in range(0, rows):
            if a == 0:
                shoot_full_row.add_effect(blocks[a][c], none, And(condition_same_colour))
            else:
                shoot_full_row.add_effect(blocks[a][c], blocks[a-1][c], And(condition_same_colour, LE(a,r)))


# changes for each row under the last column
last_column = columns-1
for u in range(1, rows):
    # this row is under the affected row
    condition_under = GT(u,r)
    condition_different_colour = And(Not(Equals(blocks[u][last_column], p)), Not(Equals(blocks[u][last_column], none)))
    # si el de baix es diferent a p i no es none, es guarda aquest a la ma i es posa p al seu lloc
    shoot_full_row.add_effect(blocks[u][last_column], p, And(condition_under, condition_different_colour))
    shoot_full_row.add_effect(hand, blocks[u][last_column], And(condition_under, condition_different_colour))


    else:
        shoot_full_row.add_effect(blocks[a][c], blocks[a-1][c], And(condition_same_colour, LE(a,r)))
# for the rows under the same column - changes

# si el de baix de l'ultima columna es de diferent color a p i no none, diem-li 'w' -> hand = w i on hi havia w hi posarem p

# hand


plotting_problem.add_action(shoot_full_row)


SyntaxError: invalid syntax (3909790719.py, line 126)